<a href="https://colab.research.google.com/github/bbanzai88/Data-Science-Repository/blob/main/Book_Writing_Crew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#This is my initial CrewAI agent.  This is a work in progress..I changed the original so I could use both colab and ollama
#together and use an initial prompt. The original version of it is from here:https://github.com/Aftabbs/Book-Writing-AI-Agent/blob/main/Book_Writing_AI_Agent.ipynb.
#I also modified it so that it only saves the actual story as output and not the agents notes/thought processes.. Each chapter is  now between 5k-10k.

In [3]:
# 1. Remove embedchain and its old chromadb/tokenizers
!pip uninstall -y embedchain chromadb tokenizers

# 2. Install only the packages you need + the correct tokenizers
!pip install \
  crewai crewai_tools langchain_community PyGithub ollama langchain \
  tokenizers==0.21.2 chromadb





Found existing installation: chromadb 1.0.15
Uninstalling chromadb-1.0.15:
  Successfully uninstalled chromadb-1.0.15
Found existing installation: tokenizers 0.21.2
Uninstalling tokenizers-0.21.2:
  Successfully uninstalled tokenizers-0.21.2
  Using cached tokenizers-0.21.2-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
ent already satisfied: packaging in /usr/local/lib/python3.11/dist-packages (from onnxruntime==1.22.0->crewai) (24.2)
Using cached tokenizers-0.21.2-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
Using cached chromadb-1.0.15-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (19.5 MB)


KeyboardInterrupt: 

In [ ]:
import transformers, tokenizers, chromadb
print(transformers.__version__, tokenizers.__version__, chromadb.__version__)



In [ ]:
%%bash
# install system deps and Ollama CLI
sudo apt-get update -y
sudo apt-get install -y pciutils

# this is Ollama’s official installer
curl https://ollama.ai/install.sh | sh


In [ ]:
import os, threading, subprocess, time

# allow any origin and bind on all interfaces
os.environ["OLLAMA_HOST"]    = "0.0.0.0:11434"
os.environ["OLLAMA_ORIGINS"] = "*"

def _serve_ollama():
    # this spins up the HTTP API at port 11434
    subprocess.Popen(["ollama", "serve"])

# start the server on a daemon thread
threading.Thread(target=_serve_ollama, daemon=True).start()

# give it a few seconds to boot
time.sleep(10)
print("✅ Ollama server should be up at localhost:11434")


In [ ]:
!ollama pull deepseek-r1:1.5b


In [ ]:
!pip install ollama langchain-ollama

In [ ]:
!pip install python-docx

In [ ]:
%%bash
# Install Ollama if you haven’t already
pip install --quiet ollama

# Start Ollama in the true background with nohup
nohup ollama serve > /content/ollama.log 2>&1 &

# Give it a moment to spin up
sleep 10

# Show the last few lines of the log to confirm it’s listening
tail -n 20 /content/ollama.log



In [ ]:
import os

# Clear any previous overrides
os.environ.pop("OLLAMA_HOST", None)
os.environ.pop("OLLAMA_ORIGINS", None)

# For CrewFlow’s nested asyncio
import nest_asyncio
nest_asyncio.apply()

# Trick OpenAI imports so they don't error
os.environ["OPENAI_API_KEY"] = ""

# Longer timeout
os.environ["LITELLM_TIMEOUT"] = "1200"

from langchain_ollama import OllamaLLM
# Initialize the shared Ollama client
ollama_llm = OllamaLLM(
    model="deepseek-r1:1.5b",
    base_url="http://127.0.0.1:11434",  # must match the server
    temperature=0
)



In [ ]:
# In a Colab cell:
!pip install --quiet nest_asyncio


In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
# Multi-Agent Book Generation with CrewAI and Ollama

"""
This notebook demonstrates how to set up a multi-agent system in CrewAI
that leverages Ollama's deepseek-r1:1.5b model for content generation.
Agents:
- SpecificationsAgent: Analyzes story requirements and maintains narrative consistency
- ProductionAgent: Generates content and implements creative changes via Ollama
- EvaluationAgent: Reviews quality and thematic resonance
- RedacteurAgent: Refines prose and maintains voice via Ollama
"""

# Install dependencies (uncomment if needed)
# !pip install crewai

# Imports
from crewai import Agent, Task, Crew, Process
import json, pprint, re, time

def robust_json_extract(raw_output):
    match = re.search(r'```json\s*(\{.*?\}|\[.*?\])\s*```', raw_output, re.DOTALL)
    if not match:
        match = re.search(r'(\{.*?\}|\[.*?\])', raw_output, re.DOTALL)
    if not match:
        raise ValueError(f"Failed to find JSON. Raw output was:\n{raw_output}")
    json_str = match.group(1)
    json_str = re.sub(r',\s*([}\]])', '\\1', json_str)
    json_str = json_str.replace('\n', ' ').replace('\r', ' ').replace('\\', '\\\\')
    try:
        return json.loads(json_str)
    except json.JSONDecodeError as e:
        json_str = re.sub(r'(\w+)\s*:', r'"\\1":', json_str)
        return json.loads(json_str)

# Initialize Agents with LiteLLM provider strings
specs_agent = Agent(
    role='SpecificationsAgent',
    goal='Extract story specs (genre, theme, tone, word_count) as JSON',
    backstory='Produce a JSON object with keys genre, theme, tone, word_count from a user idea.',
    llm='ollama/deepseek-r1:1.5b',
    verbose=True
)

prod_agent = Agent(
    role='ProductionAgent',
    goal='Generate chapter content and implement creative changes via guidelines',
    backstory='Generate full chapter text targeting 2000-4000 words based on outline and specs.',
    llm='ollama/deepseek-r1:1.5b',
    verbose=True
)

eval_agent = Agent(
    role='EvaluationAgent',
    goal='Evaluate chapter drafts and return JSON with score and feedback',
    backstory='Return JSON {"score":0-1, "feedback":"..."} evaluating chapter quality and adherence.',
    llm='ollama/deepseek-r1:1.5b',
    verbose=True
)

redac_agent = Agent(
    role='RedacteurAgent',
    goal='Polish chapter prose and maintain consistency',
    backstory='Take raw chapter text and return the refined version as plain text.',
    llm='ollama/deepseek-r1:1.5b',
    verbose=True
)

# User-provided idea
initial_idea = "A young scholar discovers an ancient prophecy that could save the kingdom."

# -- Step 1: Generate specs --
task_specs = Task(
    description=f"Extract genre, theme, tone, and word_count as JSON for idea: '{initial_idea}'",
    expected_output='json',
    agent=specs_agent
)
crew_specs = Crew(
    agents=[specs_agent],
    tasks=[task_specs],
    process=Process.sequential,
    verbose=True
)
result_specs = crew_specs.kickoff()

specs_json = robust_json_extract(result_specs.tasks_output[0].raw)

# -- Step 2: Generate chapter outline based on specs --
task_outline = Task(
    description=(
        f"Create a 10-item chapter outline as JSON array with objects having keys 'title' and 'summary' based on specs: {json.dumps(specs_json)}"
    ),
    expected_output='json',
    agent=prod_agent
)
crew_outline = Crew(
    agents=[prod_agent],
    tasks=[task_outline],
    process=Process.sequential,
    verbose=True
)
result_outline = crew_outline.kickoff()

outline_json = robust_json_extract(result_outline.tasks_output[0].raw)

# Check for dictionary format and extract chapter list accordingly
outline_list = outline_json["chapter"] if isinstance(outline_json, dict) and "chapter" in outline_json else outline_json
outline_titles = [c['title'] for c in outline_list]


# -- Step 3: Build per-chapter tasks: draft, evaluate, refine --
chapter_tasks = []
for title in outline_titles:
    chapter_tasks.extend([
        Task(
            description=(
                f"Draft full chapter titled '{title}', aiming for approximately 2,000–4,000 words "
                "to suit adult fiction pacing, based on specs and outline."
            ),
            expected_output='text',
            agent=prod_agent
        ),
        Task(
            description="Evaluate the chapter draft for score and feedback as JSON.",
            expected_output='json',
            agent=eval_agent
        ),
        Task(
            description=(
                f"Polish and refine the prose of chapter '{title}', ensuring clarity, style consistency, "
                "and adherence to tone and word-count guidelines."
            ),
            expected_output='text',
            agent=redac_agent
        )
    ])

# Execute chapter tasks in batches to avoid rate limits
batch_size = 3
auto_results = []
for i in range(0, len(chapter_tasks), batch_size):
    batch = chapter_tasks[i:i+batch_size]
    crew = Crew(
        agents=[prod_agent, eval_agent, redac_agent],
        tasks=batch,
        process=Process.sequential,
        verbose=True
    )
    time.sleep(1)
    auto_results.extend(crew.kickoff())

# -- Step 4: Display results --
pprint.pprint({
    'specs': specs_json,
    'outline_titles': outline_titles,
    'chapters_output': auto_results
})


In [ ]:
# Multi-Agent Book Generation with CrewAI and Ollama

"""
This notebook demonstrates how to set up a multi-agent system in CrewAI
that leverages Ollama's deepseek-r1:1.5b model for content generation.
Agents:
- SpecificationsAgent: Analyzes story requirements and maintains narrative consistency
- ProductionAgent: Generates content and implements creative changes via Ollama
- EvaluationAgent: Reviews quality and thematic resonance
- RedacteurAgent: Refines prose and maintains voice via Ollama
"""

# Install dependencies (uncomment if needed)
# !pip install crewai python-docx

# Imports
from crewai import Agent, Task, Crew, Process
import json, pprint, re, time
from docx import Document
from docx.shared import Pt
from docx.oxml.ns import qn

def robust_json_extract(raw_output):
    match = re.search(r'```json\s*(\{.*?\}|\[.*?\])\s*```', raw_output, re.DOTALL)
    if not match:
        match = re.search(r'(\{.*?\}|\[.*?\])', raw_output, re.DOTALL)
    if not match:
        raise ValueError(f"Failed to find JSON. Raw output was:\n{raw_output}")
    json_str = match.group(1)
    json_str = re.sub(r',\s*([}\]])', r'\1', json_str)
    json_str = json_str.replace('\n', ' ').replace('\r', ' ').replace('\\', '\\\\')
    try:
        return json.loads(json_str)
    except json.JSONDecodeError as e:
        json_str = re.sub(r'(\w+)\s*:', r'"\1":', json_str)
        return json.loads(json_str)

# Initialize Agents with LiteLLM provider strings
specs_agent = Agent(
    role='SpecificationsAgent',
    goal='Extract story specs (genre, theme, tone, word_count) as JSON',
    backstory='Produce a JSON object with keys genre, theme, tone, word_count from a user idea.',
    llm='ollama/deepseek-r1:1.5b',
    verbose=True
)

prod_agent = Agent(
    role='ProductionAgent',
    goal='Generate chapter content and implement creative changes via guidelines',
    backstory='Generate full chapter text targeting 2000-4000 words based on outline and specs.',
    llm='ollama/deepseek-r1:1.5b',
    verbose=True
)

eval_agent = Agent(
    role='EvaluationAgent',
    goal='Evaluate chapter drafts and return JSON with score and feedback',
    backstory='Return JSON {"score":0-1, "feedback":"..."} evaluating chapter quality and adherence.',
    llm='ollama/deepseek-r1:1.5b',
    verbose=True
)

redac_agent = Agent(
    role='RedacteurAgent',
    goal='Polish chapter prose and maintain consistency',
    backstory='Take raw chapter text and return the refined version as plain text.',
    llm='ollama/deepseek-r1:1.5b',
    verbose=True
)

# User-provided idea
initial_idea = "A young scholar discovers an ancient prophecy that could save the kingdom."

# -- Step 1: Generate specs --
task_specs = Task(
    description=f"Extract genre, theme, tone, and word_count as JSON for idea: '{initial_idea}'",
    expected_output='json',
    agent=specs_agent
)
crew_specs = Crew(
    agents=[specs_agent],
    tasks=[task_specs],
    process=Process.sequential,
    verbose=True
)
result_specs = crew_specs.kickoff()

specs_json = robust_json_extract(result_specs.tasks_output[0].raw)

# -- Step 2: Generate chapter outline based on specs --
task_outline = Task(
    description=(
        f"Create a 10-item chapter outline as JSON array with objects having keys 'title' and 'summary' based on specs: {json.dumps(specs_json)}"
    ),
    expected_output='json',
    agent=prod_agent
)
crew_outline = Crew(
    agents=[prod_agent],
    tasks=[task_outline],
    process=Process.sequential,
    verbose=True
)
result_outline = crew_outline.kickoff()

outline_data = robust_json_extract(result_outline.tasks_output[0].raw)
if isinstance(outline_data, dict) and 'chapter' in outline_data:
    outline_list = outline_data['chapter']
else:
    outline_list = outline_data

outline_titles = [c['title'] for c in outline_list]

# -- Step 3: Build per-chapter tasks: draft, evaluate, refine --
chapter_tasks = []
for title in outline_titles:
    chapter_tasks.extend([
        Task(
            description=(
                f"Draft full chapter titled '{title}', aiming for approximately 2,000–4,000 words "
                "to suit adult fiction pacing, based on specs and outline."
            ),
            expected_output='text',
            agent=prod_agent
        ),
        Task(
            description="Evaluate the chapter draft for score and feedback as JSON.",
            expected_output='json',
            agent=eval_agent
        ),
        Task(
            description=(
                f"Polish and refine the prose of chapter '{title}', ensuring clarity, style consistency, "
                "and adherence to tone and word-count guidelines."
            ),
            expected_output='text',
            agent=redac_agent
        )
    ])

# Execute chapter tasks in batches to avoid rate limits
batch_size = 3
auto_results = []
for i in range(0, len(chapter_tasks), batch_size):
    batch = chapter_tasks[i:i+batch_size]
    crew = Crew(
        agents=[prod_agent, eval_agent, redac_agent],
        tasks=batch,
        process=Process.sequential,
        verbose=True
    )
    time.sleep(1)
    auto_results.extend(crew.kickoff())

# -- Step 4: Save results as Word document --
doc = Document()
doc.add_heading(initial_idea, level=1)

# Optional formatting
style = doc.styles['Normal']
style.font.name = 'Times New Roman'
style._element.rPr.rFonts.set(qn('w:eastAsia'), 'Times New Roman')
style.font.size = Pt(12)

# Only use every 3rd item in auto_results, which are the refined chapters
for chapter_dict, chapter_output in zip(outline_list, auto_results[2::3]):  # Redacted outputs only
    doc.add_heading(chapter_dict['title'], level=2)
    doc.add_paragraph(str(chapter_output).strip())


output_file = 'generated_book.docx'
doc.save(output_file)
print(f"✅ Book generated and saved as '{output_file}'")


In [ ]:
# Multi-Agent Book Generation with CrewAI and Ollama

"""
This notebook demonstrates how to set up a multi-agent system in CrewAI
that leverages Ollama's deepseek-r1:1.5b model for content generation.
Agents:
- SpecificationsAgent: Analyzes story requirements and maintains narrative consistency
- ProductionAgent: Generates content and implements creative changes via Ollama
- EvaluationAgent: Reviews quality and thematic resonance
- RedacteurAgent: Refines prose and maintains voice via Ollama
"""

# Install dependencies (uncomment if needed)
# !pip install crewai python-docx

# Imports
from crewai import Agent, Task, Crew, Process
import json, pprint, re, time
from docx import Document
from docx.shared import Pt
from docx.oxml.ns import qn

def robust_json_extract(raw_output):
    match = re.search(r'```json\s*(\{.*?\}|\[.*?\])\s*```', raw_output, re.DOTALL)
    if not match:
        match = re.search(r'(\{.*?\}|\[.*?\])', raw_output, re.DOTALL)
    if not match:
        raise ValueError(f"Failed to find JSON. Raw output was:\n{raw_output}")
    json_str = match.group(1)
    json_str = re.sub(r',\s*([}\]])', r'\1', json_str)
    json_str = json_str.replace('\n', ' ').replace('\r', ' ').replace('\\', '\\\\')
    try:
        return json.loads(json_str)
    except json.JSONDecodeError:
        json_str = re.sub(r'(\w+)\s*:', r'"\1":', json_str)
        return json.loads(json_str)

# Initialize Agents
specs_agent = Agent(
    role='SpecificationsAgent',
    goal='Extract story specs (genre, theme, tone, word_count) as JSON',
    backstory='Produce a JSON object with keys genre, theme, tone, word_count from a user idea.',
    llm='ollama/deepseek-r1:1.5b',
    verbose=True
)

prod_agent = Agent(
    role='ProductionAgent',
    goal='Generate chapter content and implement creative changes via guidelines',
    backstory='Generate full chapter text targeting 2000-4000 words based on outline and specs.',
    llm='ollama/deepseek-r1:1.5b',
    verbose=True
)

eval_agent = Agent(
    role='EvaluationAgent',
    goal='Evaluate chapter drafts and return JSON with score and feedback',
    backstory='Return JSON {"score":0-1, "feedback":"..."} evaluating chapter quality and adherence.',
    llm='ollama/deepseek-r1:1.5b',
    verbose=True
)

redac_agent = Agent(
    role='RedacteurAgent',
    goal='Polish chapter prose and maintain consistency',
    backstory='Take raw chapter text and return the refined version as plain text.',
    llm='ollama/deepseek-r1:1.5b',
    verbose=True
)

# User-provided idea
initial_idea = "A young scholar discovers an ancient prophecy that could save the kingdom."

# Step 1: Generate specs
task_specs = Task(
    description=f"Extract genre, theme, tone, and word_count as JSON for idea: '{initial_idea}'",
    expected_output='json',
    agent=specs_agent
)
crew_specs = Crew(
    agents=[specs_agent],
    tasks=[task_specs],
    process=Process.sequential,
    verbose=True
)
result_specs = crew_specs.kickoff()
specs_json = robust_json_extract(result_specs.tasks_output[0].raw)

# Step 2: Generate chapter outline based on specs
task_outline = Task(
    description=(
        f"Create a 10-item chapter outline as JSON array with objects having keys 'title' and 'summary' based on specs: {json.dumps(specs_json)}"
    ),
    expected_output='json',
    agent=prod_agent
)
crew_outline = Crew(
    agents=[prod_agent],
    tasks=[task_outline],
    process=Process.sequential,
    verbose=True
)
result_outline = crew_outline.kickoff()

outline_data = robust_json_extract(result_outline.tasks_output[0].raw)
outline_list = outline_data if isinstance(outline_data, list) else outline_data.get('chapter', outline_data)
outline_titles = [c['title'] for c in outline_list]

# Step 3: Build per-chapter tasks
chapter_tasks = []
for title in outline_titles:
    chapter_tasks.extend([
        Task(
            description=(
                f"Draft full chapter titled '{title}', aiming for approximately 2,000–4,000 words "
                "to suit adult fiction pacing, based on specs and outline."
            ),
            expected_output='text',
            agent=prod_agent
        ),
        Task(
            description="Evaluate the chapter draft for score and feedback as JSON.",
            expected_output='json',
            agent=eval_agent
        ),
        Task(
            description=(
                f"Polish and refine the prose of chapter '{title}', ensuring clarity, style consistency, "
                "and adherence to tone and word-count guidelines."
            ),
            expected_output='text',
            agent=redac_agent
        )
    ])

# Step 4: Execute tasks in batches
batch_size = 3
auto_results = []
for i in range(0, len(chapter_tasks), batch_size):
    batch = chapter_tasks[i:i+batch_size]
    crew = Crew(
        agents=[prod_agent, eval_agent, redac_agent],
        tasks=batch,
        process=Process.sequential,
        verbose=True
    )
    time.sleep(1)
    auto_results.extend(crew.kickoff())

# Step 5: Save to Word document
from docx import Document
from docx.shared import Pt
from docx.oxml.ns import qn


doc = Document()
doc.add_heading(initial_idea, level=1)

style = doc.styles['Normal']
style.font.name = 'Times New Roman'
style._element.rPr.rFonts.set(qn('w:eastAsia'), 'Times New Roman')
style.font.size = Pt(12)

for chapter_dict, result in zip(outline_list, auto_results[2::3]):
    doc.add_page_break()
    doc.add_heading(chapter_dict['title'], level=2)
    if isinstance(result, tuple):
        result = result[1]  # unpack (task, result) tuple
    content = getattr(result, 'raw', str(result))
    content = re.sub(r"<think>.*?</think>", "", content, flags=re.DOTALL)
    content = re.sub(r"^\(?\\'?raw\\'?,?", "", content).strip()
    doc.add_paragraph(content.strip())

output_file = 'generated_book.docx'
doc.save(output_file)
print(f"\n📘 Book successfully generated and saved as '{output_file}'")


In [ ]:
# Multi-Agent Book Generation with CrewAI and Ollama

"""
This notebook demonstrates how to set up a multi-agent system in CrewAI
that leverages Ollama's deepseek-r1:1.5b model for content generation.
Agents:
- SpecificationsAgent: Analyzes story requirements and maintains narrative consistency
- ProductionAgent: Generates content and implements creative changes via Ollama
- EvaluationAgent: Reviews quality and thematic resonance
- RedacteurAgent: Refines prose and maintains voice via Ollama
"""

# Install dependencies (uncomment if needed)
# !pip install crewai python-docx

# Imports
from crewai import Agent, Task, Crew, Process
import json, pprint, re, time, shutil, os
from docx import Document
from docx.shared import Pt
from docx.oxml.ns import qn
from IPython.display import FileLink

def robust_json_extract(raw_output):
    match = re.search(r'```json\s*(\{.*?\}|\[.*?\])\s*```', raw_output, re.DOTALL)
    if not match:
        match = re.search(r'(\{.*?\}|\[.*?\])', raw_output, re.DOTALL)
    if not match:
        raise ValueError(f"Failed to find JSON. Raw output was:\n{raw_output}")
    json_str = match.group(1)
    json_str = re.sub(r',\s*([}\]])', r'\1', json_str)
    json_str = json_str.replace('\n', ' ').replace('\r', ' ').replace('\\', '\\\\')
    try:
        return json.loads(json_str)
    except json.JSONDecodeError:
        json_str = re.sub(r'(\w+)\s*:', r'"\1":', json_str)  # Fix missing quotes
        json_str = re.sub(r'"([^"]*?)"\s*:\s*("[^"]*?)?([,\}])', r'"\1": \2\3', json_str)  # Fix unescaped fields
        return json.loads(json_str)


# Initialize Agents
specs_agent = Agent(
    role='SpecificationsAgent',
    goal='Extract story specs (genre, theme, tone, word_count) as JSON',
    backstory='Produce a JSON object with keys genre, theme, tone, word_count from a user idea.',
    llm='ollama/deepseek-r1:1.5b',
    verbose=False
)

prod_agent = Agent(
    role='ProductionAgent',
    goal='Generate chapter content and implement creative changes via guidelines',
    backstory='Generate full chapter text targeting 2000-4000 words based on outline and specs.',
    llm='ollama/deepseek-r1:1.5b',
    verbose=False
)

eval_agent = Agent(
    role='EvaluationAgent',
    goal='Evaluate chapter drafts and return JSON with score and feedback',
    backstory='Return JSON {"score":0-1, "feedback":"..."} evaluating chapter quality and adherence.',
    llm='ollama/deepseek-r1:1.5b',
    verbose=False
)

redac_agent = Agent(
    role='RedacteurAgent',
    goal='Polish chapter prose and maintain consistency',
    backstory='Take raw chapter text and return the refined version as plain text.',
    llm='ollama/deepseek-r1:1.5b',
    verbose=False
)

# User-provided idea
initial_idea = "A young scholar discovers an ancient prophecy that could save the kingdom."

# Step 1: Generate specs
task_specs = Task(
    description=f"Extract genre, theme, tone, and word_count as JSON for idea: '{initial_idea}'",
    expected_output='json',
    agent=specs_agent
)
crew_specs = Crew(
    agents=[specs_agent],
    tasks=[task_specs],
    process=Process.sequential,
    verbose=False
)
result_specs = crew_specs.kickoff()
specs_json = robust_json_extract(result_specs.tasks_output[0].raw)

# Step 2: Generate chapter outline based on specs
task_outline = Task(
    description=(
        f"Create a 10-item chapter outline as JSON array with objects having keys 'title' and 'summary' based on specs: {json.dumps(specs_json)}"
    ),
    expected_output='json',
    agent=prod_agent
)
crew_outline = Crew(
    agents=[prod_agent],
    tasks=[task_outline],
    process=Process.sequential,
    verbose=False
)
result_outline = crew_outline.kickoff()

outline_data = robust_json_extract(result_outline.tasks_output[0].raw)
outline_list = outline_data if isinstance(outline_data, list) else outline_data.get('chapter', outline_data)
outline_titles = [c['title'] for c in outline_list]

# Step 3: Build per-chapter tasks
chapter_tasks = []
for title in outline_titles:
    chapter_tasks.extend([
        Task(
            description=(
                f"Draft full chapter titled '{title}', aiming for approximately 2,000–4,000 words "
                "to suit adult fiction pacing, based on specs and outline."
            ),
            expected_output='text',
            agent=prod_agent
        ),
        Task(
            description="Evaluate the chapter draft for score and feedback as JSON.",
            expected_output='json',
            agent=eval_agent
        ),
        Task(
            description=(
                f"Polish and refine the prose of chapter '{title}', ensuring clarity, style consistency, "
                "and adherence to tone and word-count guidelines."
            ),
            expected_output='text',
            agent=redac_agent
        )
    ])

# Step 4: Execute tasks in batches
batch_size = 3
auto_results = []
for i in range(0, len(chapter_tasks), batch_size):
    batch = chapter_tasks[i:i+batch_size]
    crew = Crew(
        agents=[prod_agent, eval_agent, redac_agent],
        tasks=batch,
        process=Process.sequential,
        verbose=False
    )
    time.sleep(1)
    auto_results.extend(crew.kickoff())

# Step 5: Save to Word document

doc = Document()
doc.add_heading(initial_idea, level=1)
doc.add_paragraph("\nTable of Contents:\n")
toc_start = doc._body._body[-1]
toc_refs = []

style = doc.styles['Normal']
style.font.name = 'Times New Roman'
style._element.rPr.rFonts.set(qn('w:eastAsia'), 'Times New Roman')
style.font.size = Pt(12)

for i, (chapter_dict, result) in enumerate(zip(outline_list, auto_results[2::3]), start=1):
    title = chapter_dict['title']
    if isinstance(result, tuple):
        result = result[1]  # unpack
    content = getattr(result, 'raw', str(result))
    content = re.sub(r"<think>.*?</think>", "", content, flags=re.DOTALL)
    content = re.sub(r"^[\(\{]?'?raw'?,?", "", content).strip()
    toc_refs.append(f"Chapter {i}: {title}")
    doc.add_page_break()
    doc.add_heading(title, level=2)
    doc.add_paragraph(content.strip())

# Insert Table of Contents manually
for ref in toc_refs:
    para = doc.paragraphs[1].insert_paragraph_before(ref)

# Updated save path
doc_path = '/content/sample_data/generated_book.docx'
os.makedirs(os.path.dirname(doc_path), exist_ok=True)
doc.save(doc_path)

print("✅ Book saved to:", doc_path)
FileLink(doc_path)


In [ ]:
# Multi-Agent Book Generation with CrewAI and Ollama

"""
This notebook demonstrates how to set up a multi-agent system in CrewAI
that leverages Ollama's deepseek-r1:1.5b model for content generation.
Agents:
- SpecificationsAgent: Analyzes story requirements and maintains narrative consistency
- ProductionAgent: Generates content and implements creative changes via Ollama
- EvaluationAgent: Reviews quality and thematic resonance
- RedacteurAgent: Refines prose and maintains voice via Ollama
"""

# Install dependencies (uncomment if needed)
# !pip install crewai python-docx

# Imports
from crewai import Agent, Task, Crew, Process
import json, pprint, re, time, shutil, os
from docx import Document
from docx.shared import Pt
from docx.oxml.ns import qn
from IPython.display import FileLink


def robust_json_extract(raw_output):
    # Attempt to extract JSON object or array
    match = re.search(r'```json\s*(\{.*?\}|\[.*?\])\s*```', raw_output, re.DOTALL)
    if not match:
        match = re.search(r'(\{.*?\}|\[.*?\])', raw_output, re.DOTALL)

    if match:
        json_str = match.group(1)
        json_str = re.sub(r',\s*([}\]])', r'\1', json_str)  # remove trailing commas
        json_str = json_str.replace('\n', ' ').replace('\r', ' ').replace('\\', '\\\\')
        try:
            return json.loads(json_str)
        except json.JSONDecodeError:
            json_str = re.sub(r'(\w+)\s*:', r'"\1":', json_str)  # fix missing quotes
            return json.loads(json_str)

    # If no valid JSON found, fallback: attempt to parse bullet-style text
    lines = raw_output.splitlines()
    spec = {}
    for line in lines:
        if "**" in line and ":" in line:
            parts = re.findall(r'\*\*(.*?)\*\*:\s*(.*)', line)
            if parts:
                key, value = parts[0]
                spec[key.lower()] = value.strip().rstrip('.')
    if spec:
        # Clean known keys to match expected schema
        return {
            "genre": spec.get("genre", ""),
            "theme": spec.get("theme", ""),
            "tone": spec.get("tone", ""),
            "word_count": int(re.search(r'\d+', spec.get("word count", "0")).group())
        }

    raise ValueError(f"Failed to find JSON. Raw output was:\n{raw_output}")

# Initialize Agents
specs_agent = Agent(
    role='SpecificationsAgent',
    goal='Extract story specs (genre, theme, tone, word_count) as JSON',
    backstory='Produce a JSON object with keys genre, theme, tone, word_count from a user idea.',
    llm='ollama/deepseek-r1:1.5b',
    verbose=False
)

prod_agent = Agent(
    role='ProductionAgent',
    goal='Generate chapter content and implement creative changes via guidelines',
    backstory='Generate full chapter text targeting 2000-4000 words based on outline and specs.',
    llm='ollama/deepseek-r1:1.5b',
    verbose=False
)

eval_agent = Agent(
    role='EvaluationAgent',
    goal='Evaluate chapter drafts and return JSON with score and feedback',
    backstory='Return JSON {"score":0-1, "feedback":"..."} evaluating chapter quality and adherence.',
    llm='ollama/deepseek-r1:1.5b',
    verbose=False
)

redac_agent = Agent(
    role='RedacteurAgent',
    goal='Polish chapter prose and maintain consistency',
    backstory='Take raw chapter text and return the refined version as plain text.',
    llm='ollama/deepseek-r1:1.5b',
    verbose=False
)

# User-provided idea
initial_idea = "A young scholar discovers an ancient prophecy that could save the kingdom."

# Step 1: Generate specs
task_specs = Task(
    description=f"Extract genre, theme, tone, and word_count as JSON for idea: '{initial_idea}'",
    expected_output='json',
    agent=specs_agent
)
crew_specs = Crew(
    agents=[specs_agent],
    tasks=[task_specs],
    process=Process.sequential,
    verbose=False
)
result_specs = crew_specs.kickoff()
specs_json = robust_json_extract(result_specs.tasks_output[0].raw)

# Step 2: Generate chapter outline based on specs
task_outline = Task(
    description=(
        f"Create a 10-item chapter outline as JSON array with objects having keys 'title' and 'summary' based on specs: {json.dumps(specs_json)}"
    ),
    expected_output='json',
    agent=prod_agent
)
crew_outline = Crew(
    agents=[prod_agent],
    tasks=[task_outline],
    process=Process.sequential,
    verbose=False
)
result_outline = crew_outline.kickoff()

outline_data = robust_json_extract(result_outline.tasks_output[0].raw)
outline_list = outline_data if isinstance(outline_data, list) else outline_data.get('chapter', outline_data)
outline_titles = [c['title'] for c in outline_list]

# Step 3: Build per-chapter tasks
chapter_tasks = []
for title in outline_titles:
    chapter_tasks.extend([
        Task(
            description=(
                f"Draft full chapter titled '{title}', aiming for approximately 2,000–4,000 words "
                "to suit adult fiction pacing, based on specs and outline."
            ),
            expected_output='text',
            agent=prod_agent
        ),
        Task(
            description="Evaluate the chapter draft for score and feedback as JSON.",
            expected_output='json',
            agent=eval_agent
        ),
        Task(
            description=(
                f"Polish and refine the prose of chapter '{title}', ensuring clarity, style consistency, "
                "and adherence to tone and word-count guidelines."
            ),
            expected_output='text',
            agent=redac_agent
        )
    ])

# Step 4: Execute tasks in batches
batch_size = 3
auto_results = []
for i in range(0, len(chapter_tasks), batch_size):
    batch = chapter_tasks[i:i+batch_size]
    crew = Crew(
        agents=[prod_agent, eval_agent, redac_agent],
        tasks=batch,
        process=Process.sequential,
        verbose=False
    )
    time.sleep(1)
    auto_results.extend(crew.kickoff())

# Step 5: Save to Word document

from docx import Document
from docx.shared import Pt
from docx.oxml.ns import qn
from IPython.display import FileLink

# Save final cleaned-up book
doc = Document()
doc.add_heading(initial_idea, level=1)

doc.add_paragraph("\nTable of Contents:\n")
toc_refs = []

style = doc.styles['Normal']
style.font.name = 'Times New Roman'
style._element.rPr.rFonts.set(qn('w:eastAsia'), 'Times New Roman')
style.font.size = Pt(12)

for i, (chapter_dict, result) in enumerate(zip(outline_list, auto_results[2::3]), start=1):
    title = chapter_dict['title']

    if isinstance(result, tuple):
        result = result[1]
    content = getattr(result, 'raw', str(result))

    content = re.sub(r"<think>.*?</think>", "", content, flags=re.DOTALL)
    content = re.sub(r"(?i)TaskOutput\(.*?raw=|expected_output=.*?\)", "", content, flags=re.DOTALL)
    content = re.sub(r"\s*agent=['\"].+?['\"]\s*", "", content)
    content = re.sub(r"[{}\[\]()<>]", "", content)
    content = re.sub(r"\\n|\\t|\\", "\n", content)
    content = re.sub(r"\n{2,}", "\n\n", content)
    content = content.strip()

    toc_refs.append(f"Chapter {i}: {title}")
    doc.add_page_break()
    doc.add_heading(f"Chapter {i}: {title}", level=2)
    doc.add_paragraph(content)

# Insert TOC at beginning
toc_para = doc.paragraphs[1]
for ref in toc_refs[::-1]:
    toc_para.insert_paragraph_before(ref)

# Save and display file
output_path = '/content/sample_data/generated_book.docx'
os.makedirs(os.path.dirname(output_path), exist_ok=True)
doc.save(output_path)
print("✅ Book saved to:", output_path)
FileLink(output_path)


In [ ]:
# Multi-Agent Book Generation with CrewAI and Ollama

"""
This notebook demonstrates how to set up a multi-agent system in CrewAI
that leverages Ollama's deepseek-r1:1.5b model for content generation.
Agents:
- SpecificationsAgent: Analyzes story requirements and maintains narrative consistency
- ProductionAgent: Generates content and implements creative changes
- EvaluationAgent: Reviews quality and thematic resonance
- RedacteurAgent: Refines prose and maintains voice
- ManagementAgent: Coordinates between agents and tracks creative flow
- ChroniqueurAgent: Documents the creative journey
- DocumentalisteAgent: Manages research and references
- DuplicationAgent: Ensures originality and prevents redundancy
- ValidationAgent: Ensures philosophical and ethical alignment
"""

# Install dependencies (uncomment if needed)
# !pip install crewai python-docx tqdm

# Imports
from crewai import Agent, Task, Crew, Process
import json, pprint, re, time, shutil, os
from docx import Document
from docx.shared import Pt
from docx.oxml.ns import qn
from IPython.display import FileLink
from tqdm.notebook import tqdm

# JSON extractor

def robust_json_extract(raw_output):
    match = re.search(r'```json\s*(\{.*?\}|\[.*?\])\s*```', raw_output, re.DOTALL)
    if not match:
        match = re.search(r'(\{.*?\}|\[.*?\])', raw_output, re.DOTALL)
    if match:
        json_str = match.group(1)
        json_str = re.sub(r',\s*([}}\]])', r'\1', json_str)
        json_str = json_str.replace('\n', ' ').replace('\r', ' ').replace('\\', '\\\\')
        try:
            return json.loads(json_str)
        except json.JSONDecodeError:
            json_str = re.sub(r'(\w+)\s*:', r'"\1":', json_str)
            return json.loads(json_str)
    lines = raw_output.splitlines()
    spec = {}
    for line in lines:
        if "**" in line and ":" in line:
            parts = re.findall(r'\*\*(.*?)\*\*:\s*(.*)', line)
            if parts:
                key, value = parts[0]
                spec[key.lower()] = value.strip().rstrip('.')
    if spec:
        return {
            "genre": spec.get("genre", ""),
            "theme": spec.get("theme", ""),
            "tone": spec.get("tone", ""),
            "word_count": int(re.search(r'\d+', spec.get("word count", "0")).group())
        }
    raise ValueError(f"Failed to find JSON. Raw output was:\n{raw_output}")

# Agent factory

def create_agent(role, goal, backstory):
    return Agent(role=role, goal=goal, backstory=backstory, llm='ollama/deepseek-r1:1.5b', verbose=False)

agents = {
    "SpecificationsAgent": create_agent('SpecificationsAgent', 'Extract story specs (genre, theme, tone, word_count)', 'Produces specs as JSON.'),
    "ProductionAgent": create_agent('ProductionAgent', 'Generate full chapter content creatively', 'Craft immersive and original story chapters.'),
    "EvaluationAgent": create_agent('EvaluationAgent', 'Evaluate quality and theme alignment', 'Rates and gives constructive chapter feedback.'),
    "RedacteurAgent": create_agent('RedacteurAgent', 'Refine chapter prose', 'Ensures clarity and consistent voice.'),
    "ManagementAgent": create_agent('ManagementAgent', 'Orchestrate agents and ensure progress', 'Coordinates narrative efforts across all roles.'),
    "ChroniqueurAgent": create_agent('ChroniqueurAgent', 'Document the creative journey', 'Keeps a log of decisions and developments.'),
    "DocumentalisteAgent": create_agent('DocumentalisteAgent', 'Support narrative with research', 'Provides relevant contextual knowledge.'),
    "DuplicationAgent": create_agent('DuplicationAgent', 'Prevent repetition and ensure novelty', 'Checks for originality and eliminates duplication.'),
    "ValidationAgent": create_agent('ValidationAgent', 'Check philosophical and ethical integrity', 'Assures alignment with deeper themes.')
}

initial_idea = "A young scholar discovers an ancient prophecy that could save the kingdom."

print("🔍 Step 1: Generating story specifications...")
spec_task = Task(
    description=f"Extract genre, theme, tone, and word_count as JSON for: '{initial_idea}'",
    expected_output='json',
    agent=agents["SpecificationsAgent"]
)
specs_result = Crew(agents=[agents["SpecificationsAgent"]], tasks=[spec_task], process=Process.sequential).kickoff()
specs_json = robust_json_extract(specs_result.tasks_output[0].raw)

print("🧠 Step 2: Generating chapter outline...")
outline_task = Task(
    description=f"Based on: {json.dumps(specs_json)}, generate a chapter outline (10-30 chapters) as JSON array with 'title' and 'summary'.",
    expected_output='json',
    agent=agents["ProductionAgent"]
)
outline_result = Crew(agents=[agents["ProductionAgent"]], tasks=[outline_task], process=Process.sequential).kickoff()
outline_data = robust_json_extract(outline_result.tasks_output[0].raw)
outline_list = outline_data if isinstance(outline_data, list) else outline_data.get('chapter', outline_data)
outline_titles = [c['title'] for c in outline_list]

print("✍️ Step 3: Generating chapters...")
chapter_tasks = []
for title in outline_titles:
    chapter_tasks.extend([
        Task(description=f"Draft chapter '{title}' based on outline and specs. Limit output to 500 words.", expected_output='text', agent=agents["ProductionAgent"]),
        Task(description=f"Evaluate draft of chapter '{title}'.", expected_output='json', agent=agents["EvaluationAgent"]),
        Task(description=f"Refine prose of chapter '{title}'.", expected_output='text', agent=agents["RedacteurAgent"]),
    ])

final_chapters = []
for i in tqdm(range(0, len(chapter_tasks), 3), desc="🛠️ Generating Chapters"):
    crew = Crew(
        agents=[agents["ProductionAgent"], agents["EvaluationAgent"], agents["RedacteurAgent"]],
        tasks=chapter_tasks[i:i+3],
        process=Process.sequential
    )
    final_chapters.append(crew.kickoff())

print("📄 Step 4: Saving final book as DOCX with TOC...")

doc = Document()
doc.add_heading(initial_idea, 0)
doc.add_paragraph("\nTable of Contents:\n")
toc_paragraph = doc.paragraphs[1]
toc_refs = []

for idx, (chapter, result) in enumerate(zip(outline_list, final_chapters), 1):
    chapter_title = chapter['title']
    if isinstance(result, tuple) and len(result) >= 3:
        refined = result[-1]
    elif isinstance(result, list) and len(result) >= 3:
        refined = result[-1]
    else:
        refined = result
    chapter_text = getattr(refined, 'raw', str(refined))
    chapter_text = re.sub(r"<think>.*?</think>", "", chapter_text, flags=re.DOTALL).strip()
    doc.add_page_break()
    doc.add_heading(f"Chapter {idx}: {chapter_title}", level=2)
    doc.add_paragraph("[Generated image placeholder]\n")
    doc.add_paragraph(chapter_text)
    toc_refs.append(f"Chapter {idx}: {chapter_title}")

for ref in reversed(toc_refs):
    toc_paragraph.insert_paragraph_before(ref)

output_path = '/content/sample_data/generated_book.docx'
os.makedirs(os.path.dirname(output_path), exist_ok=True)
doc.save(output_path)
print("✅ Book saved to:", output_path)
FileLink(output_path)


In [ ]:
# Multi-Agent Book Generation with CrewAI and Ollama
#generates something

"""
This notebook demonstrates how to set up a multi-agent system in CrewAI
that leverages Ollama's deepseek-r1:1.5b model for content generation.
Agents:
- SpecificationsAgent: Analyzes story requirements and maintains narrative consistency
- ProductionAgent: Generates content and implements creative changes
- EvaluationAgent: Reviews quality and thematic resonance
- RedacteurAgent: Refines prose and maintains voice
- ManagementAgent: Coordinates between agents and tracks creative flow
- ChroniqueurAgent: Documents the creative journey
- DocumentalisteAgent: Manages research and references
- DuplicationAgent: Ensures originality and prevents redundancy
- ValidationAgent: Ensures philosophical and ethical alignment
"""

# Install dependencies (uncomment if needed)
# !pip install crewai python-docx tqdm

# Imports
from crewai import Agent, Task, Crew, Process
import json, pprint, re, time, shutil, os
from docx import Document
from docx.shared import Pt
from docx.oxml.ns import qn
from IPython.display import FileLink
from tqdm.notebook import tqdm

# JSON extractor

def robust_json_extract(raw_output):
    match = re.search(r'```json\s*(\{.*?\}|\[.*?\])\s*```', raw_output, re.DOTALL)
    if not match:
        match = re.search(r'(\{.*?\}|\[.*?\])', raw_output, re.DOTALL)
    if match:
        json_str = match.group(1)
        json_str = re.sub(r',\s*([}}\]])', r'\1', json_str)
        json_str = json_str.replace('\n', ' ').replace('\r', ' ').replace('\\', '\\\\')
        try:
            return json.loads(json_str)
        except json.JSONDecodeError:
            json_str = re.sub(r'(\w+)\s*:', r'"\1":', json_str)
            return json.loads(json_str)
    lines = raw_output.splitlines()
    spec = {}
    for line in lines:
        if "**" in line and ":" in line:
            parts = re.findall(r'\*\*(.*?)\*\*:\s*(.*)', line)
            if parts:
                key, value = parts[0]
                spec[key.lower()] = value.strip().rstrip('.')
    if spec:
        return {
            "genre": spec.get("genre", ""),
            "theme": spec.get("theme", ""),
            "tone": spec.get("tone", ""),
            "word_count": int(re.search(r'\d+', spec.get("word count", "0")).group())
        }
    raise ValueError(f"Failed to find JSON. Raw output was:\n{raw_output}")

# Agent factory

def create_agent(role, goal, backstory):
    return Agent(role=role, goal=goal, backstory=backstory, llm='ollama/deepseek-r1:1.5b', verbose=False)

agents = {
    "SpecificationsAgent": create_agent('SpecificationsAgent', 'Extract story specs (genre, theme, tone, word_count)', 'Produces specs as JSON.'),
    "ProductionAgent": create_agent('ProductionAgent', 'Generate full chapter content creatively', 'Craft immersive and original story chapters.'),
    "EvaluationAgent": create_agent('EvaluationAgent', 'Evaluate quality and theme alignment', 'Rates and gives constructive chapter feedback.'),
    "RedacteurAgent": create_agent('RedacteurAgent', 'Refine chapter prose', 'Ensures clarity and consistent voice.'),
    "ManagementAgent": create_agent('ManagementAgent', 'Orchestrate agents and ensure progress', 'Coordinates narrative efforts across all roles.'),
    "ChroniqueurAgent": create_agent('ChroniqueurAgent', 'Document the creative journey', 'Keeps a log of decisions and developments.'),
    "DocumentalisteAgent": create_agent('DocumentalisteAgent', 'Support narrative with research', 'Provides relevant contextual knowledge.'),
    "DuplicationAgent": create_agent('DuplicationAgent', 'Prevent repetition and ensure novelty', 'Checks for originality and eliminates duplication.'),
    "ValidationAgent": create_agent('ValidationAgent', 'Check philosophical and ethical integrity', 'Assures alignment with deeper themes.')
}

initial_idea = "A young scholar discovers an ancient prophecy that could save the kingdom."

print("🔍 Step 1: Generating story specifications...")
spec_task = Task(
    description=f"Extract genre, theme, tone, and word_count as JSON for: '{initial_idea}'",
    expected_output='json',
    agent=agents["SpecificationsAgent"]
)
specs_result = Crew(agents=[agents["SpecificationsAgent"]], tasks=[spec_task], process=Process.sequential).kickoff()
specs_json = robust_json_extract(specs_result.tasks_output[0].raw)

print("🧠 Step 2: Generating chapter outline...")
outline_task = Task(
    description=f"Based on: {json.dumps(specs_json)}, generate a chapter outline (10-30 chapters) as JSON array with 'title' and 'summary'.",
    expected_output='json',
    agent=agents["ProductionAgent"]
)
outline_result = Crew(agents=[agents["ProductionAgent"]], tasks=[outline_task], process=Process.sequential).kickoff()
outline_data = robust_json_extract(outline_result.tasks_output[0].raw)
outline_list = outline_data if isinstance(outline_data, list) else outline_data.get('chapter', outline_data)
outline_titles = [c['title'] for c in outline_list]

print("✍️ Step 3: Generating chapters...")
chapter_tasks = []
for title in outline_titles:
    chapter_tasks.extend([
        Task(description=f"Draft chapter '{title}' based on outline and specs. Limit output to 500 words.", expected_output='text', agent=agents["ProductionAgent"]),
        Task(description=f"Evaluate draft of chapter '{title}'.", expected_output='json', agent=agents["EvaluationAgent"]),
        Task(description=f"Refine prose of chapter '{title}'.", expected_output='text', agent=agents["RedacteurAgent"]),
    ])

final_chapters = []
for i in tqdm(range(0, len(chapter_tasks), 3), desc="🛠️ Generating Chapters"):
    crew = Crew(
        agents=[agents["ProductionAgent"], agents["EvaluationAgent"], agents["RedacteurAgent"]],
        tasks=chapter_tasks[i:i+3],
        process=Process.sequential
    )
    try:
        result = crew.kickoff()
        final_chapters.append(result)
    except Exception as e:
        print(f"⚠️ Error in chapter generation: {e}")
        final_chapters.append("Chapter generation failed.")

print("📄 Step 4: Saving final book as DOCX with TOC...")

doc = Document()
doc.add_heading(initial_idea, 0)
doc.add_paragraph("\nTable of Contents:\n")
toc_paragraph = doc.paragraphs[1]
toc_refs = []

for idx, (chapter, result) in enumerate(zip(outline_list, final_chapters), 1):
    chapter_title = chapter['title']
    try:
        refined = result[-1] if isinstance(result, (list, tuple)) else result
        chapter_text = getattr(refined, 'raw', str(refined))
        chapter_text = re.sub(r"<think>.*?</think>", "", chapter_text, flags=re.DOTALL).strip()
    except Exception:
        chapter_text = "[Error generating chapter content.]"

    doc.add_page_break()
    doc.add_heading(f"Chapter {idx}: {chapter_title}", level=2)
    doc.add_paragraph("[Generated image placeholder]\n")
    doc.add_paragraph(chapter_text)
    toc_refs.append(f"Chapter {idx}: {chapter_title}")

for ref in reversed(toc_refs):
    toc_paragraph.insert_paragraph_before(ref)

output_path = '/content/sample_data/generated_book.docx'
os.makedirs(os.path.dirname(output_path), exist_ok=True)
doc.save(output_path)
print("✅ Book saved to:", output_path)
FileLink(output_path)


In [ ]:
#chatter removed.. get  actual story

In [ ]:
# Multi-Agent Book Generation with CrewAI and Ollama

"""
This notebook demonstrates how to set up a multi-agent system in CrewAI
that leverages Ollama's deepseek-r1:1.5b model for content generation.
Agents:
- SpecificationsAgent: Analyzes story requirements and maintains narrative consistency
- ProductionAgent: Generates content and implements creative changes
- EvaluationAgent: Reviews quality and thematic resonance
- RedacteurAgent: Refines prose and maintains voice
- ManagementAgent: Coordinates between agents and tracks creative flow
- ChroniqueurAgent: Documents the creative journey
- DocumentalisteAgent: Manages research and references
- DuplicationAgent: Ensures originality and prevents redundancy
- ValidationAgent: Ensures philosophical and ethical alignment
"""

# Install dependencies (uncomment if needed)
# !pip install crewai python-docx tqdm

# Imports
from crewai import Agent, Task, Crew, Process
import json, re, os
from docx import Document
from IPython.display import FileLink
from tqdm.notebook import tqdm

# --- Utility: JSON cleaner ---
def robust_json_extract(raw_output):
    match = re.search(r'```json\s*(\{.*?\}|\[.*?\])\s*```', raw_output, re.DOTALL)
    if not match:
        match = re.search(r'(\{.*?\}|\[.*?\])', raw_output, re.DOTALL)
    if match:
        json_str = match.group(1)
        json_str = re.sub(r',\s*([}\]])', r'\1', json_str)
        json_str = json_str.replace('\n', ' ').replace('\r', ' ').replace('\\', '\\\\')
        try:
            return json.loads(json_str)
        except json.JSONDecodeError:
            json_str = re.sub(r'(\w+)\s*:', r'"\1":', json_str)
            return json.loads(json_str)
    raise ValueError("Could not extract JSON from model output")

# --- Agent Factory ---
def create_agent(role, goal, backstory):
    return Agent(role=role, goal=goal, backstory=backstory, llm='ollama/deepseek-r1:1.5b', verbose=False)

# --- Agent Definitions ---
agents = {
    "SpecificationsAgent": create_agent('SpecificationsAgent', 'Extract story specs (genre, theme, tone, word_count)', 'Produces specs as JSON.'),
    "ProductionAgent": create_agent('ProductionAgent', 'Generate full chapter content creatively', 'Craft immersive and original story chapters.'),
    "EvaluationAgent": create_agent('EvaluationAgent', 'Evaluate quality and theme alignment', 'Rates and gives constructive chapter feedback.'),
    "RedacteurAgent": create_agent('RedacteurAgent', 'Refine chapter prose', 'Ensures clarity and consistent voice.')
}

# --- User Input ---
initial_idea = "A young scholar discovers an ancient prophecy that could save the kingdom."

# --- Step 1: Get Story Specifications ---
print("🔍 Step 1: Extracting story specs...")
spec_task = Task(
    description=f"Extract genre, theme, tone, and word_count as JSON for: '{initial_idea}'",
    expected_output='json',
    agent=agents["SpecificationsAgent"]
)
spec_crew = Crew(
    agents=[agents["SpecificationsAgent"]],
    tasks=[spec_task],
    process=Process.sequential
)
spec_result = spec_crew.kickoff()
specs = robust_json_extract(spec_result.tasks_output[0].raw)

# --- Step 2: Chapter Outline ---
print("🧠 Step 2: Generating outline...")
outline_task = Task(
    description=f"Generate a 10-30 chapter outline for story with specs: {json.dumps(specs)}. Output JSON list of objects with 'title' and 'summary'.",
    expected_output='json',
    agent=agents["ProductionAgent"]
)
outline_crew = Crew(
    agents=[agents["ProductionAgent"]],
    tasks=[outline_task],
    process=Process.sequential
)
outline_result = outline_crew.kickoff()
outline_data = robust_json_extract(outline_result.tasks_output[0].raw)
outline_list = outline_data if isinstance(outline_data, list) else outline_data.get('chapter', list(outline_data.values()))
outline_titles = [c['title'] for c in outline_list]

# --- Step 3: Generate Chapters ---
print("✍️ Step 3: Generating chapters...")
chapter_tasks = []
for title in outline_titles:
    chapter_tasks.extend([
        Task(description=f"Draft chapter '{title}' (max 500 words) with strong narrative flow. No agent chatter.", expected_output='text', agent=agents["ProductionAgent"]),
        Task(description=f"Evaluate chapter '{title}' quality.", expected_output='text', agent=agents["EvaluationAgent"]),
        Task(description=f"Refine chapter '{title}' to be immersive, clean, and in-character.", expected_output='text', agent=agents["RedacteurAgent"])
    ])

final_chapters = []
for i in tqdm(range(0, len(chapter_tasks), 3), desc="📚 Generating Chapters"):
    crew = Crew(
        agents=[agents["ProductionAgent"], agents["EvaluationAgent"], agents["RedacteurAgent"]],
        tasks=chapter_tasks[i:i+3],
        process=Process.sequential
    )
    try:
        result = crew.kickoff()
        final_chapters.append(result[-1])  # Only take refined
    except Exception as e:
        final_chapters.append(f"[Chapter generation failed: {e}]")

# --- Step 4: Save as .docx with TOC and placeholder images ---
print("📄 Step 4: Saving to Word document...")
doc = Document()
doc.add_heading(initial_idea, 0)
toc = doc.add_paragraph("\nTable of Contents:\n")
toc_refs = []

for idx, (chapter, content) in enumerate(zip(outline_list, final_chapters), 1):
    doc.add_page_break()
    title = chapter['title']
    toc_refs.append(f"Chapter {idx}: {title}")
    doc.add_heading(f"Chapter {idx}: {title}", level=2)
    doc.add_paragraph("[Generated image placeholder]\n")
    text = getattr(content, 'raw', str(content))
    cleaned = re.sub(r'<think>.*?</think>', '', text, flags=re.DOTALL).strip()
    cleaned = re.sub(r'(\{.*\}|\[.*\])', '', cleaned, flags=re.DOTALL)
    doc.add_paragraph(cleaned)

for ref in reversed(toc_refs):
    toc.insert_paragraph_before(ref)

output_path = "/content/sample_data/generated_book.docx"
os.makedirs(os.path.dirname(output_path), exist_ok=True)
doc.save(output_path)
print("✅ Book saved to:", output_path)
FileLink(output_path)


In [ ]:
# Multi-Agent Book Generation with CrewAI and OpenAI/Ollama fallback

"""
This notebook demonstrates how to set up a multi-agent system in CrewAI
that leverages Ollama (or OpenAI fallback) for content generation.
Agents:
- SpecificationsAgent: Analyzes story requirements and maintains narrative consistency
- ProductionAgent: Generates content and implements creative changes
- EvaluationAgent: Reviews quality and thematic resonance
- RedacteurAgent: Refines prose and maintains voice
"""

# Install dependencies (uncomment if needed)
# !pip install crewai python-docx tqdm openai langchain_ollama

# Imports
from crewai import Agent, Task, Crew, Process
import json, re, os
from docx import Document
from IPython.display import FileLink
from tqdm.notebook import tqdm
from langchain_ollama import OllamaLLM
from langchain.chat_models import ChatOpenAI

# --- LLM Fallback ---
def get_llm():
    try:
        return OllamaLLM(model="ollama/deepseek-r1:1.5b")
    except Exception as e:
        print("⚠️ Ollama failed, using GPT-4 fallback.")
        return ChatOpenAI(model="gpt-4", temperature=0.7)

llm_fallback = get_llm()

# --- Utility: JSON cleaner ---
def robust_json_extract(raw_output):
    match = re.search(r'```json\s*(\{.*?\}|\[.*?\])\s*```', raw_output, re.DOTALL)
    if not match:
        match = re.search(r'(\{.*?\}|\[.*?\])', raw_output, re.DOTALL)
    if match:
        json_str = match.group(1)
        json_str = re.sub(r',\s*([}\]])', r'\1', json_str)
        json_str = json_str.replace('\n', ' ').replace('\r', ' ').replace('\\', '\\\\')
        try:
            return json.loads(json_str)
        except json.JSONDecodeError:
            json_str = re.sub(r'(\w+)\s*:', r'"\1":', json_str)
            return json.loads(json_str)
    raise ValueError("Could not extract JSON from model output")

# --- Agent Factory ---
def create_agent(role, goal, backstory):
    return Agent(role=role, goal=goal, backstory=backstory, llm=llm_fallback, verbose=False)

# --- Agent Definitions ---
agents = {
    "SpecificationsAgent": create_agent('SpecificationsAgent', 'Extract story specs (genre, theme, tone, word_count)', 'Produces specs as JSON.'),
    "ProductionAgent": create_agent('ProductionAgent', 'Generate full chapter content creatively', 'Craft immersive and original story chapters.'),
    "EvaluationAgent": create_agent('EvaluationAgent', 'Evaluate quality and theme alignment', 'Rates and gives constructive chapter feedback.'),
    "RedacteurAgent": create_agent('RedacteurAgent', 'Refine chapter prose', 'Ensures clarity and consistent voice.')
}

# --- User Input ---
initial_idea = "A young scholar discovers an ancient prophecy that could save the kingdom."

# --- Step 1: Get Story Specifications ---
print("🔍 Step 1: Extracting story specs...")
spec_task = Task(
    description=f"Extract genre, theme, tone, and word_count as JSON for: '{initial_idea}'",
    expected_output='json',
    agent=agents["SpecificationsAgent"]
)
spec_crew = Crew(
    agents=[agents["SpecificationsAgent"]],
    tasks=[spec_task],
    process=Process.sequential
)
try:
    spec_result = spec_crew.kickoff()
    specs = robust_json_extract(spec_result.tasks_output[0].raw)
except Exception as e:
    raise RuntimeError(f"❌ Failed to extract specs: {e}")

# --- Step 2: Chapter Outline ---
print("🧠 Step 2: Generating outline...")
outline_task = Task(
    description=f"Generate a 2-3 chapter outline for story with specs: {json.dumps(specs)}. Output JSON list of objects with 'title' and 'summary'.",
    expected_output='json',
    agent=agents["ProductionAgent"]
)
outline_crew = Crew(
    agents=[agents["ProductionAgent"]],
    tasks=[outline_task],
    process=Process.sequential
)
try:
    outline_result = outline_crew.kickoff()
    outline_data = robust_json_extract(outline_result.tasks_output[0].raw)
    outline_list = outline_data if isinstance(outline_data, list) else outline_data.get('chapter', list(outline_data.values()))
    outline_titles = [c['title'] for c in outline_list]
except Exception as e:
    raise RuntimeError(f"❌ Failed to generate outline: {e}")

# --- Step 3: Generate Chapters ---
print("✍️ Step 3: Generating chapters...")
chapter_tasks = []
for chapter in outline_list:
    title = chapter['title']
    prompt = f"Write a fully detailed story chapter titled '{title}' (400-600 words). Stay fully in-story. No commentary, no JSON."
    chapter_tasks.extend([
        Task(description=prompt, expected_output='text', agent=agents["ProductionAgent"]),
        Task(description=f"Evaluate chapter '{title}' quality only internally.", expected_output='text', agent=agents["EvaluationAgent"]),
        Task(description=f"Refine chapter '{title}' and return only the narrative.", expected_output='text', agent=agents["RedacteurAgent"])
    ])

final_chapters = []
for i in tqdm(range(0, len(chapter_tasks), 3), desc="📚 Generating Chapters"):
    crew = Crew(
        agents=[agents["ProductionAgent"], agents["EvaluationAgent"], agents["RedacteurAgent"]],
        tasks=chapter_tasks[i:i+3],
        process=Process.sequential
    )
    try:
        result = crew.kickoff()
        final_chapters.append(result[-1])  # Only take refined
    except Exception as e:
        final_chapters.append(f"[Chapter generation failed: {e}]")

# --- Step 4: Save as .docx with TOC and placeholder images ---
print("📄 Step 4: Saving to Word document...")
doc = Document()
doc.add_heading(initial_idea, 0)
toc = doc.add_paragraph("\nTable of Contents:\n")
toc_refs = []

for idx, (chapter, content) in enumerate(zip(outline_list, final_chapters), 1):
    doc.add_page_break()
    title = chapter['title']
    toc_refs.append(f"Chapter {idx}: {title}")
    doc.add_heading(f"Chapter {idx}: {title}", level=2)
    doc.add_paragraph("[Generated image placeholder]\n")
    text = getattr(content, 'raw', str(content))
    cleaned = re.sub(r'<think>.*?</think>', '', text, flags=re.DOTALL).strip()
    cleaned = re.sub(r'(\{.*\}|\[.*\])', '', cleaned, flags=re.DOTALL)
    doc.add_paragraph(cleaned)

for ref in reversed(toc_refs):
    toc.insert_paragraph_before(ref)

output_path = "/content/sample_data/generated_book.docx"
os.makedirs(os.path.dirname(output_path), exist_ok=True)
doc.save(output_path)
print("✅ Book saved to:", output_path)
FileLink(output_path)


In [1]:
# Multi-Agent Book Generation with CrewAI and Ollama

"""
This notebook demonstrates how to set up a multi-agent system in CrewAI
that leverages Ollama (using langchain_ollama.OllamaLLM, no LiteLLM dependency)
for content generation.
"""

# --- Setup Ollama Server for Colab ---
import os, threading, subprocess, time, requests

# Disable LiteLLM entirely
os.environ["LITELLM_USE_ENV"] = "False"
os.environ["CREWAI_USE_LITELLM"] = "False"

# Clear conflicting LiteLLM/OpenAI config
os.environ.pop("OPENAI_API_KEY", None)
os.environ.pop("LITELLM_PROVIDER", None)
os.environ.pop("LITELLM_MODEL", None)
os.environ.pop("LITELLM_BASE_URL", None)

# Allow Ollama origin
os.environ["OLLAMA_HOST"] = "0.0.0.0:11434"
os.environ["OLLAMA_ORIGINS"] = "*"

def _serve_ollama():
    try:
        subprocess.Popen(["ollama", "serve"])
    except Exception as e:
        print("⚠️ Could not start Ollama:", e)

threading.Thread(target=_serve_ollama, daemon=True).start()
time.sleep(10)

try:
    r = requests.get("http://127.0.0.1:11434")
    print("✅ Ollama server running:", r.status_code)
    !ollama pull deepseek-r1:1.5b
except:
    raise RuntimeError("❌ Ollama server not responding — check install and port")

# --- Dependencies ---
from crewai import Agent, Task, Crew, Process
import json, re
from docx import Document
from IPython.display import FileLink
from tqdm.notebook import tqdm
import nest_asyncio
nest_asyncio.apply()

from langchain_ollama import OllamaLLM

# --- Shared Ollama LLM Instance ---
ollama_llm = OllamaLLM(
    model="deepseek-r1:1.5b",
    base_url="http://127.0.0.1:11434",
    temperature=0.7
)

# --- Patch Agent to use Ollama forcibly ---
from crewai.agent import Agent as CrewAgent
from types import MethodType

def patched_run_task(self, task):
    response = ollama_llm.invoke(task.description)
    class Dummy:
        raw = response
    return Dummy()

CrewAgent._run_task = MethodType(patched_run_task, CrewAgent)

# --- Patch Crew to avoid LiteLLM ---
from crewai.crew import Crew as OriginalCrew

from crewai.task import TaskOutput

from crewai.task import TaskOutput

def patched_kickoff(self):
    outputs = []
    for task in self.tasks:
        agent = task.agent
        result = agent._run_task(task)
        outputs.append(TaskOutput(
            raw=getattr(result, 'raw', str(result)),
            output=str(result),
            description=task.description,
            agent=agent.role if hasattr(agent, 'role') else str(agent)
        ))
    return outputs

OriginalCrew.kickoff = patched_kickoff

# --- Utility: JSON cleaner ---
def robust_json_extract(raw_output):
    match = re.search(r'```json\s*(\{.*?\}|\[.*?\])\s*```', raw_output, re.DOTALL)
    if not match:
        match = re.search(r'(\{.*?\}|\[.*?\])', raw_output, re.DOTALL)
    if match:
        json_str = match.group(1)
        json_str = re.sub(r',\s*([}\]])', r'\1', json_str)
        json_str = json_str.replace('\n', ' ').replace('\r', ' ').replace('\\', '\\\\')
        try:
            return json.loads(json_str)
        except json.JSONDecodeError:
            json_str = re.sub(r'(\w+)\s*:', r'"\1":', json_str)
            return json.loads(json_str)
    raise ValueError("Could not extract JSON from model output")

# --- Agent Factory ---
def create_agent(role, goal, backstory):
    return Agent(
        role=role,
        goal=goal,
        backstory=backstory,
        llm=ollama_llm,
        verbose=True,
        allow_delegation=False,
        use_litellm=False
    )

# --- Agent Definitions ---
agents = {
    "SpecificationsAgent": create_agent('SpecificationsAgent', 'Extract story specs (genre, theme, tone, word_count)', 'Produces specs as JSON.'),
    "ProductionAgent": create_agent('ProductionAgent', 'Generate full chapter content creatively', 'Craft immersive and original story chapters. Only return the final story text.'),
    "EvaluationAgent": create_agent('EvaluationAgent', 'Evaluate quality and theme alignment', 'Rates and gives constructive chapter feedback.'),
    "RedacteurAgent": create_agent('RedacteurAgent', 'Refine chapter prose', 'Ensures clarity and consistent voice.')
}

# --- User Input ---
initial_idea = "A young scholar discovers an ancient prophecy that could save the kingdom."

# --- Step 1: Get Story Specifications ---
print("🔍 Step 1: Extracting story specs...")
spec_task = Task(
    description=f"Extract genre, theme, tone, and word_count as JSON for: '{initial_idea}'",
    expected_output='json',
    agent=agents["SpecificationsAgent"]
)
spec_crew = Crew(
    agents=[agents["SpecificationsAgent"]],
    tasks=[spec_task],
    process=Process.sequential,
    use_litellm=False,
    llm_config={"llm": ollama_llm}
)
try:
    spec_result = spec_crew.kickoff()
    specs = robust_json_extract(spec_result[0].raw)
except Exception as e:
    raise RuntimeError(f"❌ Failed to extract specs: {e}")

# --- Step 2: Chapter Outline ---
print("🧠 Step 2: Generating outline...")
outline_task = Task(
    description=f"Generate a 2-3 chapter outline for story with specs: {json.dumps(specs)}. Output JSON list of objects with 'title' and 'summary'.",
    expected_output='json',
    agent=agents["ProductionAgent"]
)
outline_crew = Crew(
    agents=[agents["ProductionAgent"]],
    tasks=[outline_task],
    process=Process.sequential,
    use_litellm=False,
    llm_config={"llm": ollama_llm}
)
try:
    outline_result = outline_crew.kickoff()
    outline_data = robust_json_extract(outline_result[0].raw)
    outline_list = outline_data if isinstance(outline_data, list) else outline_data.get('chapter', list(outline_data.values()))
    outline_titles = [c['title'] for c in outline_list]
except Exception as e:
    raise RuntimeError(f"❌ Failed to generate outline: {e}")

# --- Step 3: Generate Chapters ---
print("✍️ Step 3: Generating chapters...")
chapter_tasks = []
for chapter in outline_list:
    title = chapter['title']
    prompt = f"Write a full in-story chapter titled '{title}' (400–600 words). Only return the story text. No formatting or commentary."
    chapter_tasks.extend([
        Task(description=prompt, expected_output='text', agent=agents["ProductionAgent"]),
        Task(description=f"Evaluate chapter '{title}' internally only.", expected_output='text', agent=agents["EvaluationAgent"]),
        Task(description=f"Refine prose for chapter '{title}'. Only return story.", expected_output='text', agent=agents["RedacteurAgent"])
    ])

final_chapters = []
for i in tqdm(range(0, len(chapter_tasks), 3), desc="📚 Generating Chapters"):
    crew = Crew(
        agents=[agents["ProductionAgent"], agents["EvaluationAgent"], agents["RedacteurAgent"]],
        tasks=chapter_tasks[i:i+3],
        process=Process.sequential,
        use_litellm=False,
        llm_config={"llm": ollama_llm}
    )
    try:
        result = crew.kickoff()
        final_chapters.append(result[-1])
    except Exception as e:
        final_chapters.append(f"[Chapter generation failed: {e}]")

# --- Step 4: Save as .docx with TOC and image stubs ---
print("📄 Step 4: Saving to Word document...")
doc = Document()
doc.add_heading(initial_idea, 0)
toc = doc.add_paragraph("\nTable of Contents:\n")
toc_refs = []

for idx, (chapter, content) in enumerate(zip(outline_list, final_chapters), 1):
    doc.add_page_break()
    title = chapter['title']
    toc_refs.append(f"Chapter {idx}: {title}")
    doc.add_heading(f"Chapter {idx}: {title}", level=2)
    doc.add_paragraph("[Generated image placeholder]\n")
    text = getattr(content, 'raw', str(content))
    cleaned = re.sub(r'<think>.*?</think>', '', text, flags=re.DOTALL).strip()
    cleaned = re.sub(r'(\{.*\}|\[.*\])', '', cleaned, flags=re.DOTALL)
    doc.add_paragraph(cleaned)

for ref in reversed(toc_refs):
    toc.insert_paragraph_before(ref)

output_path = "/content/sample_data/generated_book.docx"
os.makedirs(os.path.dirname(output_path), exist_ok=True)
doc.save(output_path)
print("✅ Book saved to:", output_path)
FileLink(output_path)


✅ Ollama server running: 200













🔍 Step 1: Extracting story specs...



🧠 Step 2: Generating outline...



✍️ Step 3: Generating chapters...


📚 Generating Chapters:   0%|          | 0/4 [00:00<?, ?it/s]





































📄 Step 4: Saving to Word document...
✅ Book saved to: /content/sample_data/generated_book.docx


/content/sample_data/generated_book.docx